In [21]:
from skfem import *
import numpy as np
from utils import solver_iter_krylov, solver_iter_pyamg, solver_iter_mgcg
from skfem.helpers import d, dd, ddd, dot, ddot, grad, dddot, prod
from scipy.sparse.linalg import LinearOperator, minres
from skfem.models.poisson import *
from skfem.assembly import BilinearForm, LinearForm
import datetime
import pandas as pd
import sys
import time
import matplotlib.pyplot as plt
from skfem.visuals.matplotlib import draw, plot

pi = np.pi
sin = np.sin
cos = np.cos
exp = np.exp

In [5]:
@BilinearForm
def laplace(u, v, w):
    '''
    for $(\nabla w_{h}, \nabla \chi_{h})$
    '''
    return dot(grad(u), grad(v))



@LinearForm
def f_load(v, w):
    '''
    for $(f, x_{h})$
    '''
    pix = pi * w.x[0]
    piy = pi * w.x[1]
    lu = 2 * (pi)**2 * (cos(2*pix)*((sin(piy))**2) + cos(2*piy)*((sin(pix))**2))
    llu = - 8 * (pi)**4 * (cos(2*pix)*sin(piy)**2 + cos(2*piy)*sin(pix)**2 - cos(2*pix)*cos(2*piy))
    return (epsilon**2 * llu - lu) * v

@BilinearForm
def a_load(u, v, w):
    '''
    for $a_{h}$
    '''
    return ddot(dd(u), dd(v))


@BilinearForm
def b_load(u, v, w):
    '''
    for $b_{h}$
    '''
    return dot(grad(u), grad(v))


@BilinearForm
def wv_load(u, v, w):
    '''
    for $(\nabla \chi_{h}, \nabla_{h} v_{h})$
    '''
    return dot(grad(u), grad(v))

def easy_boundary(basis):
    '''
    Input basis
    ----------------
    Return D for boundary conditions
    '''

    dofs = basis.find_dofs({
        'left': m.facets_satisfying(lambda x: x[0] == 0),
        'right': m.facets_satisfying(lambda x: x[0] == 1),
        'top': m.facets_satisfying(lambda x: x[1] == 1),
        'buttom': m.facets_satisfying(lambda x: x[1] == 0)
    })

    D = np.concatenate((dofs['left'].nodal['u'], dofs['right'].nodal['u'],
                        dofs['top'].nodal['u'], dofs['buttom'].nodal['u'],
                        dofs['left'].facet['u_n'], dofs['right'].facet['u_n'],
                        dofs['top'].facet['u_n'], dofs['buttom'].facet['u_n']))
    return D

In [6]:
def exact_u(x, y):
    return (sin(pi * x) * sin(pi * y))**2

def dexact_u(x, y):
    dux = 2 * pi * cos(pi * x) * sin(pi * x) * sin(pi * y)**2
    duy = 2 * pi * cos(pi * y) * sin(pi * x)**2 * sin(pi * y)
    return dux, duy

def ddexact(x, y):
    duxx = 2*pi**2*cos(pi*x)**2*sin(pi*y)**2 - 2*pi**2*sin(pi*x)**2*sin(pi*y)**2
    duxy = 2*pi*cos(pi*x)*sin(pi*x)*2*pi*cos(pi*y)*sin(pi*y)
    duyx = duxy
    duyy = 2*pi**2*cos(pi*y)**2*sin(pi*x)**2 - 2*pi**2*sin(pi*y)**2*sin(pi*x)**2
    return duxx, duxy, duyx, duyy

@Functional
def L2uError(w):
    x, y = w.x
    return (w.w - exact_u(x, y))**2

def get_DuError(basis, u):
    duh = basis.interpolate(u).grad
    x = basis.global_coordinates().value
    dx = basis.dx  # quadrature weights
    dux, duy = dexact_u(x[0], x[1])
    return np.sqrt(np.sum(((duh[0] - dux)**2 + (duh[1] - duy)**2) * dx))

def get_D2uError(basis, u):
    dduh = basis.interpolate(u).hess
    x = basis.global_coordinates().value  # coordinates of quadrature points [x, y]
    dx = basis.dx  # quadrature weights
    duxx, duxy, duyx, duyy = ddexact(x[0], x[1])
    return np.sqrt(
        np.sum(((dduh[0][0] - duxx)**2 + (dduh[0][1] - duxy)**2 +
                (dduh[1][1] - duyy)**2 + (dduh[1][0] - duyx)**2) * dx))

In [11]:
L2_list = []
Du_list = []
D2u_list = []
h_list = []
epu_list = []
m = MeshTri()

for i in range(1, 7):

    m.refine()
    element = {'w': ElementTriP1(), 'u': ElementTriMorley()}
    basis = {variable: InteriorBasis(m, e, intorder=4)
        for variable, e in element.items()} 

    K1 = asm(laplace, basis['w'])
    f1 = asm(f_load, basis['w'])

    wh = solve(*condense(K1, f1, D=m.boundary_nodes()), solver=solver_iter_krylov(Precondition=True))

    K2 = asm(b_load, basis['u'])
    f2 = asm(wv_load, basis['w'], basis['u']) * wh
    uh0 = solve(*condense(K2, f2, D=basis['u'].find_dofs()), solver=solver_iter_krylov(Precondition=True))

    U = basis['u'].interpolate(uh0).value

    L2u = np.sqrt(L2uError.assemble(basis['u'], w=U))
    Du = get_DuError(basis['u'], uh0)
    H1u = Du + L2u
    D2u = get_D2uError(basis['u'], uh0)
    H2u = Du + L2u + D2u
    epu = np.sqrt(epsilon**2 * D2u**2 + Du**2)
    h_list.append(m.param())
    Du_list.append(Du)
    L2_list.append(L2u)
    D2u_list.append(D2u)
    epu_list.append(epu)

hs = np.array(h_list)
L2s = np.array(L2_list)
Dus = np.array(Du_list)
D2us = np.array(D2u_list)
epus = np.array(epu_list)
H1s = L2s + Dus
H2s = H1s + D2us
print('epsilon =', epsilon)
print('  h    L2u   H1u   H2u   epu')
for i in range(H2s.shape[0] - 1):
    print(
        '2^-' + str(i + 2),
        ' {:.2f}  {:.2f}  {:.2f}  {:.2f}'.format(-np.log2(L2s[i + 1] / L2s[i]),
                                                -np.log2(H1s[i + 1] / H1s[i]),
                                                -np.log2(H2s[i + 1] / H2s[i]),
                                                -np.log2(epus[i + 1] / epus[i])))

epsilon = 1e-05
  h    L2u   H1u   H2u   epu
2^-2  1.43  0.77  0.45  0.70
2^-3  2.29  1.66  0.81  1.61
2^-4  2.31  1.89  0.94  1.87
2^-5  2.14  1.97  0.98  1.96
2^-6  2.04  1.99  1.00  1.99


In [42]:
L2_list = []
Du_list = []
D2u_list = []
h_list = []
epu_list = []
m = MeshTri()

m.refine(5)
element = {'w': ElementTriP2(), 'u': ElementTriMorley()}
basis = {variable: InteriorBasis(m, e, intorder=4)
    for variable, e in element.items()} 

K1 = asm(laplace, basis['w'])
f1 = asm(f_load, basis['w'])

wh = solve(*condense(K1, f1, D=m.boundary_nodes()), solver=solver_iter_krylov(Precondition=True))

In [43]:
W = basis['w'].interpolate(wh).value

L2u = np.sqrt(L2uError.assemble(basis['w'], w=W))
Du = get_DuError(basis['w'], wh)
H1u = Du + L2u

In [44]:
basis['w'].interpolate(wh)

DiscreteField(value=array([[-6.28795781e-07,  6.13060528e-07, -6.28795781e-07,
         6.13060528e-07,  2.29007083e-06, -6.73690893e-07],
       [-3.41928636e-06, -3.45851851e-06, -9.43226781e-06,
         5.01591754e-05,  7.67956879e-06, -8.35133122e-06],
       [-9.43226781e-06,  5.01591754e-05, -3.41928636e-06,
        -3.45851851e-06,  7.67956879e-06, -8.35133122e-06],
       ...,
       [ 6.00225605e-02,  6.71316464e-02,  6.00225605e-02,
         6.71316464e-02,  6.74271755e-02,  5.27966205e-02],
       [ 5.48138233e-01,  5.93286316e-01,  5.69254083e-01,
         5.45228118e-01,  5.69254083e-01,  5.45228118e-01],
       [ 5.69254083e-01,  5.45228118e-01,  5.69254083e-01,
         5.45228118e-01,  5.48138233e-01,  5.93286316e-01]]), grad=array([[[ 6.70192007e-05,  5.06410204e-04,  2.19252659e-04,
          1.79602311e-04,  3.33684936e-04, -6.60557189e-05],
        [-1.75173673e-03,  1.72204792e-04,  6.12663270e-04,
         -4.90358205e-03, -2.28192177e-03,  1.31038202e-03],
     

In [37]:
D2u = get_D2uError(basis['w'], wh)

TypeError: 'NoneType' object is not subscriptable

In [ ]:
H2u = Du + L2u + D2u
epu = np.sqrt(Du**2)
h_list.append(m.param())
Du_list.append(Du)
L2_list.append(L2u)
D2u_list.append(D2u)
epu_list.append(epu)

hs = np.array(h_list)
L2s = np.array(L2_list)
Dus = np.array(Du_list)
D2us = np.array(D2u_list)
epus = np.array(epu_list)
H1s = L2s + Dus
H2s = H1s + D2us
print('epsilon =', epsilon)
print('  h    L2u   H1u   H2u   epu')
for i in range(H2s.shape[0] - 1):
    print(
        '2^-' + str(i + 2),
        ' {:.2f}  {:.2f}  {:.2f}  {:.2f}'.format(-np.log2(L2s[i + 1] / L2s[i]),
                                                -np.log2(H1s[i + 1] / H1s[i]),
                                                -np.log2(H2s[i + 1] / H2s[i]),
                                                -np.log2(epus[i + 1] / epus[i])))

In [34]:
L2_list = []
Du_list = []
D2u_list = []
h_list = []
epu_list = []
m = MeshTri()

for i in range(1, 7):

    m.refine()
    element = {'w': ElementTriP2(), 'u': ElementTriMorley()}
    basis = {variable: InteriorBasis(m, e, intorder=4)
        for variable, e in element.items()} 

    K1 = asm(laplace, basis['w'])
    f1 = asm(f_load, basis['w'])

    wh = solve(*condense(K1, f1, D=m.boundary_nodes()), solver=solver_iter_krylov(Precondition=True))

    W = basis['w'].interpolate(wh).value

    L2u = np.sqrt(L2uError.assemble(basis['w'], w=W))
    Du = get_DuError(basis['w'], wh)
    H1u = Du + L2u
    D2u = get_D2uError(basis['w'], wh)
    H2u = Du + L2u + D2u
    epu = np.sqrt(Du**2)
    h_list.append(m.param())
    Du_list.append(Du)
    L2_list.append(L2u)
    D2u_list.append(D2u)
    epu_list.append(epu)

hs = np.array(h_list)
L2s = np.array(L2_list)
Dus = np.array(Du_list)
D2us = np.array(D2u_list)
epus = np.array(epu_list)
H1s = L2s + Dus
H2s = H1s + D2us
print('epsilon =', epsilon)
print('  h    L2u   H1u   H2u   epu')
for i in range(H2s.shape[0] - 1):
    print(
        '2^-' + str(i + 2),
        ' {:.2f}  {:.2f}  {:.2f}  {:.2f}'.format(-np.log2(L2s[i + 1] / L2s[i]),
                                                -np.log2(H1s[i + 1] / H1s[i]),
                                                -np.log2(H2s[i + 1] / H2s[i]),
                                                -np.log2(epus[i + 1] / epus[i])))

epsilon = 1e-05
  h    L2u   H1u   H2u   epu
2^-2  0.38  0.10  -0.62  0.03
2^-3  -0.19  -0.25  -1.29  -0.26
2^-4  -0.06  -0.08  -1.08  -0.08
2^-5  -0.02  -0.02  -1.01  -0.02
2^-6  -0.00  -0.01  -1.00  -0.01


In [32]:
L2_list = []
Du_list = []
D2u_list = []
h_list = []
epu_list = []
m = MeshTri()

for i in range(1, 7):

    m.refine()
    element = {'w': ElementTriP1(), 'u': ElementTriMorley()}
    basis = {variable: InteriorBasis(m, e, intorder=4)
        for variable, e in element.items()}

    K1 = asm(laplace, basis['w'])
    f1 = asm(f_load, basis['w'])

    wh = solve(*condense(K1, f1, D=m.boundary_nodes()), solver=solver_iter_krylov(Precondition=True))

#     K2 = asm(b_load, basis['u'])
#     f2 = asm(wv_load, basis['w'], basis['u']) * wh
#     uh0 = solve(*condense(K2, f2, D=basis['u'].find_dofs()), solver=solver_iter_krylov(Precondition=True))

    U = basis['w'].interpolate(wh).value

    L2u = np.sqrt(L2uError.assemble(basis['w'], w=U))
    Du = get_DuError(basis['w'], wh)
    H1u = Du + L2u
#     D2u = get_D2uError(basis['w'], wh)
#     H2u = Du + L2u + D2u
#     epu = np.sqrt(epsilon**2 * D2u**2 + Du**2)
    h_list.append(m.param())
    Du_list.append(Du)
    L2_list.append(L2u)
#     D2u_list.append(D2u)
#     epu_list.append(epu)

hs = np.array(h_list)
L2s = np.array(L2_list)
Dus = np.array(Du_list)
H1s = L2s + Dus
print('epsilon =', epsilon)
print('  h    L2u   H1u ')
for i in range(H2s.shape[0] - 1):
    print(
        '2^-' + str(i + 2),
        ' {:.2f}  {:.2f} '.format(-np.log2(L2s[i + 1] / L2s[i]),
                                                -np.log2(H1s[i + 1] / H1s[i])))

epsilon = 1e-05
  h    L2u   H1u 
2^-2  0.78  0.31 
2^-3  1.75  0.94 
2^-4  1.93  1.00 
2^-5  1.98  1.01 
2^-6  2.00  1.01 
